## TI3160TU: Natural Language Processing - Basic Text Preprocessing Lab

In this hands-on lab we will dive into the main preprocessing steps when preparing data for use in various NLP techniques. Particularly, we are going to see how to implement and run:
1. **Regular Expressions (Regex)**
2. **Word Tokenization & Sentence Segmentation**
3. **Lemmatization and Stemming**
4. **Basic text similarity metrics based on Hamming Distance, Levenshtein Distance, and Jaccard Index**


For the purposes of this lab, we will load some data from Reddit and use it to implement and run the abovementioned steps. Particularly, the dataset includes all the comments made by Reddit users on the /r/TUDelft subreddit between June 2022 and December 2022.
**Make sure that the data file (comments_TUDelft.ndjson) is in the same directory as the Jupyter Notebook. In case you are using online platforms such as Collab, make sure to first upload the data file before running the cells below.**

### 0. Preparation and Loading Dataset

In [1]:
# make sure that we have installed the required packages for this lab
!pip install tldextract # package that is used to extract domains from URLs

import nltk
nltk.download('punkt')

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: jupyter-server 2.0.0 has a non-standard dependency specifier jupyter-core!=~5.0,>=4.12. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of jupyter-server or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/szannettou/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
# we need the library json as the reddit data is stored in line-delimited json objects
# (one json object in each line, with each line representing a Reddit comment)
import json

# function to load all comment data into a list of strings
# Input: the path of the file including our data
# Output: a list of strings including the body of the Reddit comments
def load_reddit_comment_data(data_directory):

    comments_data = [] # list object that will store the loaded Reddit comments

    # we first open the file that includes our dataset
    with open(data_directory, 'r', encoding='utf-8') as f:
        # iterate the file, reading it line by line
        for line in f:
            # load the data petraining to a line into a json object in memory
            data = json.loads(line)

            # append the comment
            comments_data.append(data['body'])

    # the method returns all the loaded Reddit comments
    return comments_data

# our data is stored in this file
data_dir = './comments_TUDelft.ndjson'
# lets load our dataset into memory
reddit_data = load_reddit_comment_data(data_dir)
print("Successfully loaded Reddit comments! Our dataset includes %d Reddit comments!" %len(reddit_data))

Successfully loaded Reddit comments! Our dataset includes 2263 Reddit comments!


## 1. Regular Expressions

Regular expressions are a powerful tool allowing to perform text operations based on a formal language (syntax).
Regular expressions can be used to define a variety of rules that allows us to solve various tasks including:
* Searching for patterns in text and extracting data from documents
* Manipulating text (e.g., substituting text)


## 1.1 Searching for patterns in text and extracting data using Regular Expressions

In [4]:
# For working with regular expresssions we are going to use the "re" library.
# The library allow us to define and run regular expressions in our datasets.
import re

# function that performs a search pattern in our reddit dataset
# INPUTS: the regular expression pattern and the input dataset
# OUTPUT: a list with all Reddit comments matching the regular expression
def search_pattern(pattern, data):

    # list to store our matching comments
    matches = []

    # iterate through the comments
    for comment in data:

        # search for our regex pattern
        if re.search(regex_pattern, comment):

            # lets store the comment in our list
            matches.append(comment)

    # return the matching comments
    return matches

# lets try to do a simple search by finding all comments mentioning "delft"
regex_pattern = '[dD]elft' # we define our regex pattern and we want to make sure that we either match delft or Delft

# we run our function to extract all comments mentioning Delft/delft
comments_mentioning_delft = search_pattern(regex_pattern, reddit_data)
comments_mentioning_delft[:10]

['I got a 40 in IB (international baccalaureate), from maths A HL I got a 5 (1 mark off of a 6) and from physics HL a 6\n\nIn the Delft ranking I was 445\n\nKeep in mind the selection procedure tests were right around the time I had mocks internal exams at school so I was super busy with that and barely had time to study for the selection procedure.',
 'Same, I am an IBer as well. But is it really that you only need 24 for IB to get into Delft if pass the selection procedures?',
 "There should be another button allowing you to recharge your account in the main menu. Screenshot: https://imgur.com/a/HyRrCxM \nMaybe trying to log in from another device will work. If it doesn't you can also go to the printing shop in front of delft centraal",
 'You should contact the TU Delft student ambassadors. They’ll connect you with someone',
 'Aerospace is a highly competitive BSc. You need to meet the minimum requirements for application. Even when you meet the requirements, you will still have to g

The result of the **re.search** operation is more than a Boolean operator. It provides more information about the match
For instance, it can give us information on where the match happened in the string
Lets see an example with the comments including Delft

In [5]:
# repeat the same search in the first document that we already knows that includes the term delft/Delft
match = re.search(regex_pattern, comments_mentioning_delft[0])

# print the start and end of the match in the first document
print("The start of the match is on character %d, while the end of the match is in the character %d" %(match.start(), match.end()))


The start of the match is on character 126, while the end of the match is in the character 131


Note that the **re.search** method provides information only for the first match.
So the questions that arises is how we deal with a document that has multiple occurrences of the pattern we are searching for?
In cases where we want to find information about all the occurrences, we need to use the **re.finditer** method.

In [6]:
# import the pandas library that helps us store and visualize data in a tabular format
import pandas as pd
pd.set_option('display.max_colwidth', 100)


def search_all_pattern(pattern, data):

    # list to store our data
    all_matches = []

    # keep track of the comment index to store it in the output (acts as a unique identifier for each comment in this simple example)
    i = 0

    # iterate through the comments
    for comment in data:

        # search for our regex pattern
        matches = re.finditer(pattern, comment)

        # iterate through all matches in the comment
        for match in matches:
            # lets store all and the matches start and end in a json object in our list (one json object for each match)
            all_matches.append({'comment_index': i, 'comment': comment, "match_start": match.start(), "match_end": match.end()})

        # increase the comment index
        i+=1
    # return the matching comments and the start/end of each match in a pandas DataFrame
    return pd.DataFrame(all_matches)

# run our function to get all the matches in our Reddit dataset
all_delft_matches = search_all_pattern(regex_pattern, reddit_data)

# print how many matches we found
print("Overall we find %d matches for Delft/delft in our dataset! " %(all_delft_matches.shape[0]))

# lets also print some examples (the first two rows in our pandas DataFrame)
all_delft_matches[:20]

Overall we find 437 matches for Delft/delft in our dataset! 


,comment_index,comment,match_start,match_end
0,6,"I got a 40 in IB (international baccalaureate), from maths A HL I got a 5 (1 mark off of a 6) an...",126,131
1,7,"Same, I am an IBer as well. But is it really that you only need 24 for IB to get into Delft if p...",86,91
2,19,There should be another button allowing you to recharge your account in the main menu. Screensho...,245,250
3,28,You should contact the TU Delft student ambassadors. They’ll connect you with someone,26,31
4,29,Aerospace is a highly competitive BSc. You need to meet the minimum requirements for application...,209,214
5,29,Aerospace is a highly competitive BSc. You need to meet the minimum requirements for application...,531,536
6,32,It's all on the TUDelft website. To enter you just have to have solid maths knowledge (verify yo...,18,23
7,35,you can find this information [here](https://www.tudelft.nl/onderwijs/toelating-en-aanmelding/bs...,51,56
8,41,If you can get the housing service you absolutely should take it! There is a huge housing proble...,112,117
9,46,"Thank you for the advise. Yes, it is the ""TU Delft’s housing portal"" which as I understand it wo...",45,50


Observe for example that in this output we have comments with multiple appearances of our matching pattern. For instance, lets look at the comment with index 129 (it has two entries in our DataFrame, indicating that our pattern appeared twice in the comment)

In [7]:
reddit_data[129]

'You open Google, find the TU Delft Aerospace website and start reading.\n\nhttps://www.tudelft.nl/en/onderwijs/opleidingen/bachelors/aerospace-engineering/bsc-aerospace-engineering\n\nThe website is quite extensive and enrollment procedure is detailed there. Please put some effort into your question next time.'

### Use case: Extract all URLs that are mentioned in our Reddit dataset and find the most popular domains

Lets assume that we want to study what links (i.e., URLs) are shared by Reddit users on the /r/TUDelft subreddit. Lets see how we can use the power of regular expressions to achieve this goal. We are going to use regular expressions to extract all URLs and then use an off-the-shelf tool to convert URLs to domains and find the most popular domains

In [8]:
# function that extracts all URLs that appear in a list of strings
# Input: a dataset in the form of a list of strings
# Output: a list of URLs that appear in the dataset
def extract_urls(data):
    # simple regular expression for URLs. does not cover all possible URL formats :)
    regex = '(https?://\S+)'

    # list to store the results
    urls = []

    # iterate through the dataset
    for comment in data:

        # the findall method returns all matches in a list of strings
        matches = re.findall(regex, comment)

        # store all the matches in our list
        urls.extend(matches)

    # return the result
    return urls

# use our function to extract the URLs in our data
all_urls = extract_urls(reddit_data)

# print some stats/results
print("Overall we find %d URLs in our dataset!" %(len(all_urls)))
print("Some examples below...")
print(all_urls[:10])

Overall we find 155 URLs in our dataset!
Some examples below...
['https://imgur.com/a/HyRrCxM', 'https://www.tudelft.nl/en/education/admission-and-application/bsc-international-diploma/admission-requirements/diploma-with-additional-requirements#c106230', 'https://www.stud.nl/)', 'https://www.tudelft.nl/onderwijs/toelating-en-aanmelding/bsc-nederlands-diploma/1-toelatingseisen#c39206).', 'https://www.tudelft.nl/onderwijs/toelating-en-aanmelding/msc-international-diploma/required-documents#c425773](https://www.tudelft.nl/onderwijs/toelating-en-aanmelding/msc-international-diploma/required-documents#c425773)', 'https://www.tudelft.nl/en/onderwijs/opleidingen/bachelors/aerospace-engineering/bsc-aerospace-engineering', 'https://www.tudelft.nl/en/education/admission-and-application/bsc-international-diploma/application-procedure', 'https://oras.nl/post/opening-hours-2/', 'https://www.reddit.com/r/TUDelft/comments/sv34r7/are_tuition_fees_paid_yearly_or_every_semester/hxfd7jc/):', 'https://www

In [9]:
import tldextract # library that will help us convert URLs to domains
from collections import Counter # library that provides some easy-to-use tools to perform various count operations

# function that extracts the domain given a URL
# Input: a string corresponding to a URL
# Output: a string corresponding to the domain of the given URL
def extract_domain(url):
    return tldextract.extract(url).registered_domain

# use list comprehension to run our extract_domain function for each URL in our dataset
all_domains = [extract_domain(x) for x in all_urls]

# use the Counter library to extract the 10 most common domains in our dataset along with the number of appearances (list of tuples)
Counter(all_domains).most_common(10)

[('tudelft.nl', 74),
 ('reddit.com', 14),
 ('cloudfront.net', 6),
 ('whatsapp.com', 5),
 ('belastingdienst.nl', 3),
 ('wikipedia.org', 3),
 ('doorstroommatrix.nl', 3),
 ('github.com', 2),
 ('oweedelft.com', 2),
 ('ns.nl', 2)]

From this basic analysis using regular expressions we can observe that the most popular domains by users commenting on the /r/TUDelft subreddit are unsuprisingly, the TUDelft website (74 appearances), followed by URLs that point to other resources in the Reddit platform (14 appearances).

### **Exercise:** Given this particular Reddit dataset, use regular expressions to find all posts talking about Architecture, Aerospace, or TPM/TBM. Comment on which is the most popular faculty/topic among these three and inspect the matching comments to see how well your regular expressions work!


**Tip:** You can re-use the *search_all_pattern* method that we implemented above!

In [ ]:
### Insert your code here

## 1.2 Manipulating text using regular expressions (we focus on substitution)

In many cases, we want to normalize the text before processing it. To achieve this, we can use regular expressions to manipulate text and make it more uniform. Some examples include:
1. Converting British English to US English (or vice-versa depending on use-case)
2. Changing the case of some characters (e.g., delft -> Delft)

Lets see an example on how we can perform text substitution using regular expressions in Python using the simplified example of delft -> Delft



In [11]:
# first lets see how many times each word appears in our dataset
num_matches_delft = search_all_pattern('delft', reddit_data).shape[0]
num_matches_Delft = search_all_pattern('Delft', reddit_data).shape[0]
print("Numbers of occurrences for delft=%d and Delft=%d" %(num_matches_delft, num_matches_Delft))

Numbers of occurrences for delft=170 and Delft=267


In [12]:
# a function that substitutes the regex matches with the new word
# Input: a regular expression to match and the new word that corresponds to the replaced regex and the data to run the substitutions on
# Output: a list of the data with the replacements as described by the regex and the new word
def substitute_using_regex(regex, new_word, data):

    # list to hold the result (i.e., the list of substituted comments)
    results = []

    # iterate through the comments in our dataset
    for comment in data:

        # we use the re.sub method to replace text matching the regex with the new_word for the current comment
        substituted_text = re.sub(regex, new_word, comment)

        # store the substituted text into our results list
        results.append(substituted_text)

    # return the results
    return results

delft_pattern = 'delft' # we want to match everything that has delft so that we convert it to Delft
new_delft_word = 'Delft' # this is the string that we want to have in the normalized dataset

# call our function to replace all instances of delft to Delft
reddit_data_substituted = substitute_using_regex(delft_pattern, new_delft_word, reddit_data)


Now, Lets verify our results using our previously defined *search_all_pattern* (returns all matches given a specific regex)

In [13]:
num_matches_delft = search_all_pattern('delft', reddit_data_substituted).shape[0]
num_matches_Delft = search_all_pattern('Delft', reddit_data_substituted).shape[0]
print("Numbers of occurrences for delft=%d and Delft=%d" %(num_matches_delft, num_matches_Delft))

Numbers of occurrences for delft=0 and Delft=437


**Use case:** In many applications, when doing analysis on large corpora of text we want to preprocess the data and remove some parts. For instance, we might want to remove URLs from our data. Here, we will see how we can use a similar approach to the above to remove all URLs from our dataset

In [14]:
regex_url = '(https?://\S+)' # simple regular expression for URLs

# lets check how many URL apperances we have in our raw dataset (before preprocessing)
print("Number of URL matches in the raw dataset = %d" %(search_all_pattern(regex_url, reddit_data).shape[0]))


Number of URL matches in the raw dataset = 155


In [15]:

# here, we are re-using the same function as before. we provide our simple regex to match the URLs and for new word we simply give the empty string
# this corresponds to deleting the URLs, as the URLs will be substituted with an empty string
reddit_data_no_url = substitute_using_regex(regex_url, '', reddit_data)

In [16]:
print("Number of URL matches in the preprocessed dataset = %d" %(search_all_pattern(regex_url, reddit_data_no_url).shape[0]))


Number of URL matches in the preprocessed dataset = 0


**Use case #2:** Using a similar approach, we can preprocess the dataset so that all dates (yyyy-mm-dd) are replaced by their respective year


In [17]:
# simple regex to match the dates in the form of yyyy-mm-dd
date_regex = "\d{4}-\d{2}-\d{2}"

# lets first extract all the matches with dates
date_matches = search_all_pattern(date_regex, reddit_data)

#We need a substitution function that extracts the year based on the input, as we dont want to simply have a specific year but be adaptable
# Input: a date (in the form of yyyy-mm-dd)
# Output: a string corresponding to the date's year
def replace_year(date):
    # Extract the year from the date ( first 4 characters) and return it
    return date.group(0)[:4]

# function to substitute the dates. here we use the re.sub and note that instead of providing a "new_word" we provide a function (replace_year)
# that dictates what exactly should be the output based on the input
# Input: a string corresponding to the Reddit comment
# Output: a substituted comment where dates are replaced by their year
def sub_dates(text):
    return re.sub(date_regex, replace_year, text)

# we use the pandas map function to run the function "sub_dates" to all the rows in our dataframe. we use as input the data in the "comment" column
# we store the output in the "comment_without_date" column
date_matches['comment_without_date'] = date_matches['comment'].map(sub_dates)

# lets inspect manually the changes by comparing the "comment" and "comment_without_date" columns
date_matches

,comment_index,comment,match_start,match_end,comment_without_date
0,155,I'll refer to [a post I made a while back](https://www.reddit.com/r/TUDelft/comments/sv34r7/are_...,1047,1057,I'll refer to [a post I made a while back](https://www.reddit.com/r/TUDelft/comments/sv34r7/are_...
1,323,I will be messaging you in 1 day on [**2022-11-09 20:20:41 UTC**](http://www.wolframalpha.com/in...,39,49,I will be messaging you in 1 day on [**2022 20:20:41 UTC**](http://www.wolframalpha.com/input/?i...
2,323,I will be messaging you in 1 day on [**2022-11-09 20:20:41 UTC**](http://www.wolframalpha.com/in...,103,113,I will be messaging you in 1 day on [**2022 20:20:41 UTC**](http://www.wolframalpha.com/input/?i...
3,323,I will be messaging you in 1 day on [**2022-11-09 20:20:41 UTC**](http://www.wolframalpha.com/in...,534,544,I will be messaging you in 1 day on [**2022 20:20:41 UTC**](http://www.wolframalpha.com/input/?i...


### **Exercise:** Using regular expressions, implement a function that removes all numbers from the comments included in the Reddit dataset


In [ ]:
### Insert your code here

## 2. Word Tokenization & Sentence Segmentation

Word tokenization refers to the procedure of segmenting a string into words. Here, we will see how we can perform word tokenization using Python libraries.
There are many libraries that have implemented many off-the-shelf word tokenization methods:
1. NLTK
2. Gensim library
3. SpaCy
4. Stanford CoreNLP


In [21]:
from nltk.tokenize import word_tokenize
from gensim.utils import tokenize
import en_core_web_sm # python3 -m spacy download en_core_web_sm
# We load an NLP model available on SpaCy (can be used for various tasks)
spacy_nlp = en_core_web_sm.load()
# function to perform simple word tokenization using the NLTK library
# Input: a string of text
# Output: a list of strings (each string corresponds to a word)
def tokenize_text_nltk(text):
    return word_tokenize(text) # we use the word_tokenize method from NLTK


# function to perform simple word tokenization using the Gensim library
# Input: a string of text
# Output: a list of strings (each string corresponds to a word)
def tokenize_text_gensim(text):
    return list(tokenize(text)) # we use the tokenize method from Gensim

# function to perform simple word tokenization using the SpaCy library
# Input: a string of text
# Output: a list of strings (each string corresponds to a word)
def tokenize_text_spacy(text):
    # we run the spacy model
    doc = spacy_nlp(text)
    # we extract the words after tokenization and return them
    return [token.text for token in doc]

# use the functions to tokenize an example of a post in our Reddit data
tokenized_nltk = tokenize_text_nltk(reddit_data[0])
tokenized_gensim = tokenize_text_gensim(reddit_data[0])
tokenized_spacy = tokenize_text_spacy(reddit_data[0])

# print the results to check how the text is tokenized
print("Lets see how a specific Reddit post will get tokenized using these Libraries...")
print("Raw post = %s" %(reddit_data[0]))
print("NLTK Tokenization = %s Num of words = %d" %(str(tokenized_nltk), len(tokenized_nltk)))
print("Gensim Tokenization = %s Num of words = %d" %(str(tokenized_gensim), len(tokenized_gensim)))
print("SpaCy Tokenization = %s Num of words = %d" %(str(tokenized_spacy), len(tokenized_spacy)))


Lets see how a specific Reddit post will get tokenized using these Libraries...
Raw post = I’d say they give that after jan 13 or when everyone’s place has been secured
NLTK Tokenization = ['I', '’', 'd', 'say', 'they', 'give', 'that', 'after', 'jan', '13', 'or', 'when', 'everyone', '’', 's', 'place', 'has', 'been', 'secured'] Num of words = 19
Gensim Tokenization = ['I', 'd', 'say', 'they', 'give', 'that', 'after', 'jan', 'or', 'when', 'everyone', 's', 'place', 'has', 'been', 'secured'] Num of words = 16
SpaCy Tokenization = ['I', '’d', 'say', 'they', 'give', 'that', 'after', 'jan', '13', 'or', 'when', 'everyone', '’s', 'place', 'has', 'been', 'secured'] Num of words = 17


### Examine the output of these three word tokenization methods. What do you observe?

Each word tokenization method deals with the problem of word tokenization slighly different. For instance, the Gensim implementation removes all punctuation and numbers before tokenizing the document. Also, the NLTK and SpaCy tokenization methods behave differently for words or parts of the document that include punctuation such as "I'd". Overall, depending on the intended use case and analysis, one should choose which of these methods better suits their needs and if none is well-suited for the intended use, then we implement our own word tokenization method (e.g., using regular expressions or word dictionaries).

### Usually, word tokenization is used in conjuction with other basic text normalization operations such as stopword removal.

Stop words are common words that do not carry much meaningful information and are often removed in pre-processing steps of text mining tasks. Here's how you can remove stop words using three popular libraries: NLTK, SpaCy, and Gensim.



In [22]:
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import remove_stopwords
nltk.download('stopwords', quiet=True)

# remove stopwords using NLTK
# Input: a string of text
# Output: a list of words corresponding to the input text without stop words
def remove_stopwords_nltk(text):
    # load the list of stopwords from NLTK
    stop_words = set(stopwords.words('english'))

    # split the text into word using the word tokenization function we implmentated before
    word_tokens = tokenize_text_nltk(text)

    # return the words as long as they are not in our stop words list
    return [word for word in word_tokens if not word in stop_words]


# remove stopwords using SpaCy
# Input: a string of text
# Output: a list of words corresponding to the input text without stop words
def remove_stopwords_spacy(text):
    # process the text using SpaCy
    doc = spacy_nlp(text)

    # return the words as long as they are not in SpaCy's stopword list
    return [token.text for token in doc if not token.is_stop]

# remove stopwords using Gensim's method
# Input: a string of text
# Output: a string without any stop words
def remove_stopwords_gensim(text):
    return tokenize_text_nltk(remove_stopwords(text))

toy_example = "This is a sample sentence, showing off the stop words removal."

print(remove_stopwords_nltk(toy_example))
print(remove_stopwords_spacy(toy_example))
print(remove_stopwords_gensim(toy_example))

['This', 'sample', 'sentence', ',', 'showing', 'stop', 'words', 'removal', '.']
['sample', 'sentence', ',', 'showing', 'stop', 'words', 'removal', '.']
['This', 'sample', 'sentence', ',', 'showing', 'stop', 'words', 'removal', '.']


What do you observe from all these various methods to perform stop word removal? What is the effect of the case of the text? Play with some toy examples to see what is the result for various instances of the same word (e.g., This vs this).

### Sentence Segmentation

Sentence segmentation is the problem of dividing a string of written language into its component sentences. The idea here seems very trivial, and it is a trivial task for a human since we understand the structure of each sentence. However, for a machine, this task is not straightforward. Here, we are gonna see how we can use the NLTK and SpaCy libraries to perform sentence segmentation.

We do not consider Gensim here because Gensim's *split_sentences* is a rather basic and crude method of sentence segmentation. It essentially just splits on periods, without more advanced logic for dealing with periods in the middle of sentences (like abbreviations or decimal numbers). This is a naive approach and will fail in cases where periods are used in floating numbers or abbreviations.

In [23]:
from nltk.tokenize import sent_tokenize

# function to perform sentence segmentation using the NLTK library
# Input: a string of text
# Output: a list of strings (each string corresponds to a sentence)
def sentence_segm_nltk(text):
    return sent_tokenize(text) # we use the sent_tokenize method from NLTK

# function to perform simple sentence segmentation using the SpaCy library
# Input: a string of text
# Output: a list of strings (each string corresponds to a sentence)
def sentence_segm_spacy(text):
    # we run the spacy model
    doc = spacy_nlp(text)
    # we extract the words after tokenization and return them
    return [sent for sent in doc.sents]

# lets try the NLTK Sentence segmentation method using a toy example
toy_example = "Dr. Smith graduated from the University of Amsterdam. His Ph.D. thesis was inspiring. He now works at OpenAI Corp. and earns $1200.50 per day."
sentence_segm_nltk(toy_example)

['Dr. Smith graduated from the University of Amsterdam.',
 'His Ph.D. thesis was inspiring.',
 'He now works at OpenAI Corp. and earns $1200.50 per day.']

In [24]:
# lets try the SpaCy Sentence segmentation method using a toy example
sentence_segm_spacy(toy_example)

[Dr. Smith graduated from the University of Amsterdam.,
 His Ph.D. thesis was inspiring.,
 He now works at OpenAI Corp. and earns $1200.50 per day.]

## 3. Lemmatization and Stemming

In natural language processing, **stemming** and **lemmatization** are techniques for reducing words to their root form. They are used in text preprocessing steps to group words having the same meaning but different representations.

* **Stemming:** It reduces the word to its stem or root form. For example, 'jumps', 'jumped', and 'jumping' are stemmed to 'jump'. It is a rudimentary rule-based process of stripping the suffixes ("ing", "ly", "es", "s" etc) from a word.

* **Lemmatization:** It reduces the word to its base form. The difference between stemming and lemmatization is, lemmatization considers the context and converts the word to its meaningful base form, whereas stemming just removes the last few characters, which may result in incorrect meanings and spelling errors.


Here, we are going to see how we can implement in Python, lemmatization and stemming using the NLTK library.

In [25]:
from nltk.stem import PorterStemmer

# function that performs stemming using the Porter Stemmer algorithm
# Input: a string of text
# Output: a list of stemmed words extracted after performing tokenization and stemming using the Porter Stemmer Algorithm
def stem_text(text):
    # create an instance of PorterStemmer (simple stemming algorithm)
    stemmer = PorterStemmer()

    # split the text into words using our already defined word tokenization methods (here we use NLTK's tokenization)
    words = tokenize_text_nltk(text)

    # stem each word using our PorterStemmer instance
    stemmed_words = [stemmer.stem(word) for word in words]

    # return the list of the stemmed words
    return stemmed_words

# stem an example document
toy_document = "This was not the map we found in Billy Bones’s chest, but an accurate copy, complete in all things-names and heights and soundings-with the single exception of the red crosses and the written notes."
print(stem_text(toy_document))

['thi', 'wa', 'not', 'the', 'map', 'we', 'found', 'in', 'billi', 'bone', '’', 's', 'chest', ',', 'but', 'an', 'accur', 'copi', ',', 'complet', 'in', 'all', 'things-nam', 'and', 'height', 'and', 'soundings-with', 'the', 'singl', 'except', 'of', 'the', 'red', 'cross', 'and', 'the', 'written', 'note', '.']


In [26]:
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet', quiet=True)

# function that performs lemmatization using the WordNet model available via NLTK
# Input: a string of text
# Output: a list of lemmatized words extracted after performing tokenization and lemmatization using the Porter Stemmer Algorithm
def lemmatize_text(text):
    # create an instance of the WordNet Lemmatizer
    lemmatizer = WordNetLemmatizer()

    # split the text into words using our already defined word tokenization methods (here we use NLTK's tokenization)
    words = tokenize_text_nltk(text)

    # lemmatize each word using our WordNetLemmatizer instance
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

    # return the list of the lemmatized words
    return lemmatized_words

print(lemmatize_text(toy_document))

['This', 'wa', 'not', 'the', 'map', 'we', 'found', 'in', 'Billy', 'Bones', '’', 's', 'chest', ',', 'but', 'an', 'accurate', 'copy', ',', 'complete', 'in', 'all', 'things-names', 'and', 'height', 'and', 'soundings-with', 'the', 'single', 'exception', 'of', 'the', 'red', 'cross', 'and', 'the', 'written', 'note', '.']


## 4. Basic text similarity metrics based on Hamming Distance, Levenshtein Distance, and Jaccard Index

In [27]:
# function to calculate the hamming distance between two strings
# recall that the hamming distance is simply the number of characters that mismatch
# Recall that the Hamming distances works only for strings that have equal length
# Input: two strings
# Output: an integer corresponding to the Hamming distance between the two strings
def hamming_distance(string1, string2):
    # check to ensure that the two strings have the same length. raise an error if they do not have the same length
    if len(string1) != len(string2):
        raise ValueError("Undefined for sequences of unequal length.")

    return sum(el1 != el2 for el1, el2 in zip(string1, string2))


# function that calculates the jaccard index (score) between two sets of words
# recall that the jaccard index is the length of the intersection divided by the length of the union of the two sets
# Input: two sets corresponding to words
# Ouput: a score between 0 and 1 corresponding to the Jaccard index of the two sets
def jaccard_index(set1, set2):

    # calculate the length of the intersection of the two sets
    intersection = len(set(set1).intersection(set(set2)))

    # calculate the length of the union between the two sets
    union = len(set(set1)) + len(set(set2)) - intersection

    # calculate jaccard score and return it
    return intersection / union


# function that calculates the Levenshtein Distance by creating the Levenshtein Distance matrix
# Input: two strings
# Output: an integer corresponding to the Levenshtein Distance of the two strings
def levenshtein_distance(string1, string2):

    # define the number of rows and columns in our table (length of string + 1)
    size_x = len(string1) + 1
    size_y = len(string2) + 1

    # create the matrix (list of lists)
    matrix = [[0] * size_y for _ in range(size_x)]

    # initiliaze the first row (with 0 to size_x)
    for x in range(size_x):
        matrix[x][0] = x

    # initialize the first column (with 0 to size_y)
    for y in range(size_y):
        matrix[0][y] = y

    # we iterate all indices of the table and we fill the distance matrix
    for x in range(1, size_x):
        for y in range(1, size_y):

            # if the characters match
            if string1[x-1] == string2[y-1]:
                # we select the minimum value between the insert/copy/delete operations
                #Note: since the characters match this is equivalant to simply using the copy operation
                # i.e., matrix[x][y] = matrix[x-1][y-1]
                matrix[x][y] = min(
                    matrix[x-1][y] + 1, # insert operation
                    matrix[x-1][y-1], # copy operation
                    matrix[x][y-1] + 1 # delete operation
                )

            # if the characters do not match
            else:
                # we select the minimum value between the insert/substitute/delete operations
                matrix[x][y] = min(
                    matrix[x-1][y] + 1, # insert operation
                    matrix[x-1][y-1] + 1, # substitute operation
                    matrix[x][y-1] + 1 # delete operation
                )
    # return the bottom right element in our matrix that corresponds to the Levenshtein Distance of the two strings
    return matrix[size_x - 1][size_y - 1]


In [28]:
# lets test these distance metrics with toy examples
doc1 = ["the", "quick", "brown", "fox", "jumps", "over", "the", "lazy", "dog"]
doc2 = ["the", "lazy", "dog", "is", "jumped", "over", "by", "the", "quick", "brown", "fox"]


print("Hamming Distance between compture and computer = %d" %(hamming_distance("compture", "computer")))
print("Levenshtein Distance between Saturday and Sunday = %d" %(levenshtein_distance("Saturday", "Sunday")))
print("The Jaccard index between %s and %s = %.2f" %(str(doc1), str(doc2), jaccard_index(doc1, doc2)))


Hamming Distance between compture and computer = 4
Levenshtein Distance between Saturday and Sunday = 3
The Jaccard index between ['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog'] and ['the', 'lazy', 'dog', 'is', 'jumped', 'over', 'by', 'the', 'quick', 'brown', 'fox'] = 0.64


## **Use case:** Finding similar documents in a dataset

Using the Reddit dataset, and a specific document, lets try to find the closest post in our dataset using one of the similarity metrics (for the purposes of this use case lets use Jaccard Index)

In [29]:
# function to find the most similar document in a corpus given a target document
# Input: a string corresponding to a target document and a list of strings in our dataset to be used for finding the most similar document
# Output: a string that have the highest jaccard index across all documents and its jaccard index score
def find_closest_document(document, documents):

    # Tokenize the document so that we have a list of words. For this use case we use the SpaCy tokenization defined above
    words_in_document = tokenize_text_spacy(document.lower())

    # initialize the closest_document variable and the highest jaccard index (we set it to something very low)
    closest_document = ""
    highest_jaccard_index = -1

    # iterate through the documents we want to search in
    for doc in documents:

        # tokenize each document (.lower ensures that everything is in lowercase)
        words_in_doc = tokenize_text_spacy(doc.lower())

        # calculate jaccard index of this specific document and the document we are searching with
        index = jaccard_index(words_in_document, words_in_doc)

        # if its higher that the already existing highest_jaccard_index, update the variables holding the highest jaccard index and the closest document
        if index > highest_jaccard_index:
            highest_jaccard_index = index
            closest_document = doc

    # return the closest document based on the jaccard index
    return closest_document, highest_jaccard_index

# run the function with a test document
find_closest_document("The university moocs are a good resource for the exam", reddit_data)

('The pre university calculus and physics moocs by tudelft are a great resource to practise the concepts needed for the entrance exam.',
 0.36363636363636365)

## **Exercise:** Finding similar documents in a dataset 2

Implement and run a function called "find_closest_document_levenshtein" that finds the closest post in our dataset using the Levensthein Distance metric implemented above

In [ ]:
# Insert your code here

Here, re-run the find_closest methods with various examples of documents to observe the results and assess how the following aspects affect the results:
1. Inclusion of stop words in the data
2. Length of the documents and how this affect the distance metrics and the Jaccard Distance

In [ ]:
# Insert your code here

## TI3160TU: Natural Language Processing - Basic Text Preprocessing Lab -- END